In [6]:
import os
import pandas as pd
import numpy as np
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC'

avgt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\figures\daily\CoAg-CoAg'
mint_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\figures\daily\CoAg-CoAg'
maxt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\figures\daily\CoAg-CoAg'

avgt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\tables\daily\CoAg-CoAg'
mint_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\tables\daily\CoAg-CoAg'
maxt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\tables\daily\CoAg-CoAg'

os.chdir(path)

xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"
csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\vessel.csv'

In [8]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [9]:
sns.set_style('dark')
n=0
for index,rows in meta.iterrows():
    try:
        print('Grabbing CoAg data for ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin',
            filename=csv)
        coag = pd.read_csv(csv)
        coag = coag.reset_index()
    except Exception as e:
        print('Something went wrong while calling the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Cleaning CoAg data for ' + str(meta.ID[n]))
        coag.columns.values[0] = 'station'
        coag.columns.values[1] = 'date'
        coag.columns.values[2] = 'CoAg_tmean'
        coag.columns.values[3] = 'CoAg_tmax'
        coag.columns.values[4] = 'CoAg_tmin'

        coag['date'] = pd.to_datetime(coag.date)
        coag['julian'] = coag.date.dt.strftime('%j')

        coag.CoAg_tmean = coag.CoAg_tmean.astype(float)
        coag.CoAg_tmax = coag.CoAg_tmax.astype(float)
        coag.CoAg_tmin = coag.CoAg_tmin.astype(float)

        coag.CoAg_tmean = 32 + (coag.CoAg_tmean * (9/5))
        coag.CoAg_tmax = 32 + (coag.CoAg_tmax * (9/5))
        coag.CoAg_tmin = 32 + (coag.CoAg_tmin * (9/5))

        coag=coag[:-1]
    except Exception as e:
        print('Something went wrong while cleaning the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Calculating normals for ' + str(meta.ID[n]))
        normal = coag.groupby(['julian']).mean().reset_index()
    except Exception as e:
        print('Something went wrong while calculating normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Merging the observed data and normals for ' + str(meta.ID[n]))
        anomaly = coag.merge(normal,'outer',left_on='julian',right_on='julian',suffixes=('_observed','_normal'))
    except Exception as e:
        print('Something went wrong while merging the observed data and normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Cleaning up the merged dataset for ' + str(meta.ID[n]))
        avgt_only = ['CoAg_tmax_observed','CoAg_tmin_observed','CoAg_tmax_normal','CoAg_tmin_normal']
        maxt_only = ['CoAg_tmean_observed','CoAg_tmin_observed','CoAg_tmean_normal','CoAg_tmin_normal']
        mint_only = ['CoAg_tmean_observed','CoAg_tmax_observed','CoAg_tmean_normal','CoAg_tmax_normal']

        avgt_anomaly = anomaly.drop(avgt_only, axis=1)
        maxt_anomaly = anomaly.drop(maxt_only, axis=1)
        mint_anomaly = anomaly.drop(mint_only, axis=1)

        avgt_anomaly = avgt_anomaly[['date','julian','CoAg_tmean_observed','CoAg_tmean_normal']]
        maxt_anomaly = maxt_anomaly[['date','julian','CoAg_tmax_observed','CoAg_tmax_normal']]
        mint_anomaly = mint_anomaly[['date','julian','CoAg_tmin_observed','CoAg_tmin_normal']]

        avgt_anomaly['anomaly'] = avgt_anomaly['CoAg_tmean_observed'] - avgt_anomaly['CoAg_tmean_normal']
        maxt_anomaly['anomaly'] = maxt_anomaly['CoAg_tmax_observed'] - maxt_anomaly['CoAg_tmax_normal']
        mint_anomaly['anomaly'] = mint_anomaly['CoAg_tmin_observed'] - mint_anomaly['CoAg_tmin_normal']

        avgt_anomaly['date'] = pd.to_datetime(avgt_anomaly.date)
        maxt_anomaly['date'] = pd.to_datetime(maxt_anomaly.date)
        mint_anomaly['date'] = pd.to_datetime(mint_anomaly.date)
        
        avgt_anomaly['year'] = avgt_anomaly.date.dt.year
        maxt_anomaly['year'] = maxt_anomaly.date.dt.year
        mint_anomaly['year'] = mint_anomaly.date.dt.year
        
        avgt_anomaly['month'] = avgt_anomaly.date.dt.month
        maxt_anomaly['month'] = maxt_anomaly.date.dt.month
        mint_anomaly['month'] = mint_anomaly.date.dt.month
    except Exception as e:
        print('Something went wrong while cleaning up the merged dataset for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Storing daily anomaly datasets for ' + str(meta.ID[n]))
        avgt_anomaly.to_csv(avgt_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_avgt_daily_anomaly.csv')
        maxt_anomaly.to_csv(maxt_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_maxt_daily_anomaly.csv')
        mint_anomaly.to_csv(mint_tabpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_mint_daily_anomaly.csv')
    except Exception as e:
        print('Something went wrong while storing the daily anomaly datasets for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Visualizing and storing daily anomaly figures for ' + str(meta.ID[n]))
        avgt_ax = sns.catplot(x='year',y='anomaly', data=avgt_anomaly,kind='swarm',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Daily Average Temperature Anomaly by year')
        plt.ylabel('Daily deviation from Daily Average Temperature Normals (°F)')
        plt.xlabel('Year')
        plt.savefig(avgt_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_avgt_anomaly.png',bbox_inches='tight')
        plt.clf()
        maxt_ax = sns.catplot(x='year',y='anomaly',data=maxt_anomaly,kind='swarm',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Daily Maximum Temperature Anomaly by year')
        plt.ylabel('Daily deviation from Daily Maximum Temperature Normals (°F)')
        plt.xlabel('Year')
        plt.savefig(maxt_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_maxt_anomaly.png',bbox_inches='tight')
        plt.clf()
        mint_ax = sns.catplot(x='year',y='anomaly',data=mint_anomaly,kind='swarm',palette='colorblind',height=8.27,aspect=11.7/8.27)
        plt.title(str(meta.ID[n]) + ' - Daily Minimum Temperature Anomaly by year')
        plt.ylabel('Daily deviation from Daily Minimum Temperature Normals (°F)')
        plt.xlabel('Year')
        plt.savefig(mint_figpath + r'\CoAg-CoAg_' + str(meta.ID[n]) + '_mint_anomaly.png',bbox_inches='tight')
        plt.close()
    except Exception as e:
        print('Something went wrong while visualizing the anomaly data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    n=n+1
        
        

Grabbing CoAg data for AKR02
Cleaning CoAg data for AKR02
Calculating normals for AKR02
Merging the observed data and normals for AKR02
Cleaning up the merged dataset for AKR02
Storing daily anomaly datasets for AKR02
Visualizing and storing daily anomaly figures for AKR02
Grabbing CoAg data for ALT01
Cleaning CoAg data for ALT01
Calculating normals for ALT01
Merging the observed data and normals for ALT01
Cleaning up the merged dataset for ALT01
Storing daily anomaly datasets for ALT01
Visualizing and storing daily anomaly figures for ALT01
Grabbing CoAg data for AVN01
Cleaning CoAg data for AVN01
Calculating normals for AVN01
Merging the observed data and normals for AVN01
Cleaning up the merged dataset for AVN01
Storing daily anomaly datasets for AVN01
Visualizing and storing daily anomaly figures for AVN01
Grabbing CoAg data for BNV01
Cleaning CoAg data for BNV01
Calculating normals for BNV01
Merging the observed data and normals for BNV01
Cleaning up the merged dataset for BNV01
S

Grabbing CoAg data for HOT01
Cleaning CoAg data for HOT01
Calculating normals for HOT01
Merging the observed data and normals for HOT01
Cleaning up the merged dataset for HOT01
Storing daily anomaly datasets for HOT01
Visualizing and storing daily anomaly figures for HOT01
Grabbing CoAg data for HXT01
Cleaning CoAg data for HXT01
Calculating normals for HXT01
Merging the observed data and normals for HXT01
Cleaning up the merged dataset for HXT01
Storing daily anomaly datasets for HXT01
Visualizing and storing daily anomaly figures for HXT01
Grabbing CoAg data for HYD01
Cleaning CoAg data for HYD01
Calculating normals for HYD01
Merging the observed data and normals for HYD01
Cleaning up the merged dataset for HYD01
Storing daily anomaly datasets for HYD01
Visualizing and storing daily anomaly figures for HYD01
Grabbing CoAg data for HYK02
Cleaning CoAg data for HYK02
Calculating normals for HYK02
Merging the observed data and normals for HYK02
Cleaning up the merged dataset for HYK02
S

Grabbing CoAg data for SCM01
Cleaning CoAg data for SCM01
Calculating normals for SCM01
Merging the observed data and normals for SCM01
Cleaning up the merged dataset for SCM01
Storing daily anomaly datasets for SCM01
Visualizing and storing daily anomaly figures for SCM01
Grabbing CoAg data for SLD01
Cleaning CoAg data for SLD01
Calculating normals for SLD01
Merging the observed data and normals for SLD01
Cleaning up the merged dataset for SLD01
Storing daily anomaly datasets for SLD01
Visualizing and storing daily anomaly figures for SLD01
Grabbing CoAg data for SLT01
Cleaning CoAg data for SLT01
Calculating normals for SLT01
Merging the observed data and normals for SLT01
Cleaning up the merged dataset for SLT01
Storing daily anomaly datasets for SLT01
Visualizing and storing daily anomaly figures for SLT01
Grabbing CoAg data for STG01
Cleaning CoAg data for STG01
Calculating normals for STG01
Merging the observed data and normals for STG01
Cleaning up the merged dataset for STG01
S

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

...

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>

<Figure size 842.4x595.44 with 0 Axes>